## Initial Sanity checks

In [0]:
%sql
SELECT * FROM silver.crm_products
WHERE category_id NOT IN (SELECT DISTINCT category_id FROM silver.erp_product_category)

## Check for duplicates

In [0]:
%sql
SELECT * FROM(
SELECT * , count(1) over(partition by product_number)  as checkduplicates
from silver.crm_products)
WHERE checkduplicates > 1


# Transformation Logic

In [0]:
%sql
SELECT 
row_number() over(order by cr.product_id) as product_key,
cr.product_id, cr.product_number,cr.product_name,cr.category_id,
       er.subcategory,er.maintenance_flag ,cr.product_cost,cr.product_line,cr.start_date
       --,cr.end_date
FROM silver.crm_products cr
LEFT JOIN silver.erp_product_category er
ON cr.category_id = er.category_id
WHERE cr.end_date is NULL -- Filter out all historical data


In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType , IntegerType
from pyspark.sql.functions import trim, col

In [0]:
query="""
SELECT 
row_number() over(order by cr.product_id) as product_key,
cr.product_id, cr.product_number,cr.product_name,cr.category_id,
       er.subcategory,er.maintenance_flag ,cr.product_cost,cr.product_line,cr.start_date
       --,cr.end_date
FROM silver.crm_products cr
LEFT JOIN silver.erp_product_category er
ON cr.category_id = er.category_id
WHERE cr.end_date is NULL -- Filter out all historical data
"""
df = spark.sql(query)
display(df)

## Final checks

In [0]:
null_stats = (
    df.select([
        F.count(F.when(F.col(c).isNull(), c)).alias(c)
        for c in df.columns
    ])
)

display(null_stats)

In [0]:
df.filter(col("subcategory").isNull()).display()

# Writing Gold Table

In [0]:
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable("workspace.gold.dim_products")

# Sanity checks of Gold Table

In [0]:
%sql
SELECT * FROM gold.dim_products

In [0]:
%sql
SELECT *, count FROM(
SELECT product_number, count(1) OVER(partition by product_number) as count
FROM gold.dim_products
)
WHERE count > 1
